<a href="https://colab.research.google.com/github/Rodiyaturrahmi/ProsesKlasifikasi/blob/main/ProsesKlasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import string
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 28.6 MB/s 


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = 'drive/MyDrive/sentimen/dataset_tweet_sentimen.csv'
data = pd.read_csv(dataset)
data

,tweet,Sentimen
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif
...,...,...
995,"Kalian tak percaya Covid ada, himbauan pemerin...",Positif
996,"Berat perjuangan pemerintah Aceh, ketika rakya...",Positif
997,Anjuran pemerintah tidak boleh berpegangan tan...,Positif
998,"gue bingung data covid naik terus, yang salah ...",Positif


tahap pre-processing data

In [ ]:
#cleaning Text
def cleaning(text):
  text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")                #menghilangkan tab, new line, black slice
  text = text.encode('ascii', 'replace').decode('ascii')                                               #menghilangkan emoticon, chinese word,dll
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\s+)"," ", text).split())                        #menghilangkan mention, link, hashtag
  return text.replace("http://", " ").replace("https://", " ")                                         #menghilangkan url

data['clean'] = data['tweet'].apply(lambda x: cleaning(x))
data.head(5)

,tweet,Sentimen,clean
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif,bit.ly/2L6CcbB? | Seharusnya saat ini Pemerin...
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif,Klo kata gw Pemerintah tuh lagi menerapkan Her...
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif,Saat ini yang bisa saya lakukan hanya menyiapk...
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif,"emosi bgt, lg kondisi begini gue disuruh liput..."
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif,Bang gimna pemerintah mau peduli RRC urus ABK ...


In [ ]:
def remove(text):
  text = re.sub(r"\d+", "", text)                                #menghilangkan number
  text = text.translate(str.maketrans("","",string.punctuation)) #menghilangkan punctuation
  text = text.strip()                                            #menghilangkan whitespace leading & trailing
  text = re.sub('\s+', ' ', text)                                #menghilangkan multipe whitespace into single shitespace
  text = re.sub(r"\b[^a-zA-Z]\b", " ", text)                     #menghilangkan single char
  text = re.sub(r'\b\w{1,2}\b', ' ', text)
  return text

data['cleantweet'] = data['clean'].apply(lambda x: remove(x))
data.head(5)

,tweet,Sentimen,clean,cleantweet
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif,bit.ly/2L6CcbB? | Seharusnya saat ini Pemerin...,bitlyLCcbB Seharusnya saat ini Pemerintah Lamp...
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Klo kata Pemerintah tuh lagi menerapkan Herd...
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif,Saat ini yang bisa saya lakukan hanya menyiapk...,Saat ini yang bisa saya lakukan hanya menyiapk...
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif,"emosi bgt, lg kondisi begini gue disuruh liput...",emosi bgt kondisi begini gue disuruh liputan...
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif,Bang gimna pemerintah mau peduli RRC urus ABK ...,Bang gimna pemerintah mau peduli RRC urus ABK ...


In [ ]:
#Tokenization
def tokenization(text):
  return word_tokenize(text)

data['TOKENIZATION'] = data['cleantweet'].apply(lambda x: tokenization(x.lower()))
data.head(5)

,tweet,Sentimen,clean,cleantweet,TOKENIZATION
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif,bit.ly/2L6CcbB? | Seharusnya saat ini Pemerin...,bitlyLCcbB Seharusnya saat ini Pemerintah Lamp...,"[bitlylccbb, seharusnya, saat, ini, pemerintah..."
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Klo kata Pemerintah tuh lagi menerapkan Herd...,"[klo, kata, pemerintah, tuh, lagi, menerapkan,..."
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif,Saat ini yang bisa saya lakukan hanya menyiapk...,Saat ini yang bisa saya lakukan hanya menyiapk...,"[saat, ini, yang, bisa, saya, lakukan, hanya, ..."
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif,"emosi bgt, lg kondisi begini gue disuruh liput...",emosi bgt kondisi begini gue disuruh liputan...,"[emosi, bgt, kondisi, begini, gue, disuruh, li..."
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif,Bang gimna pemerintah mau peduli RRC urus ABK ...,Bang gimna pemerintah mau peduli RRC urus ABK ...,"[bang, gimna, pemerintah, mau, peduli, rrc, ur..."


In [ ]:
#stop Removal
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(["yg", "dg", "dgn", "ny", "d", "klo",
                      'kalo', 'amp', 'sih', 'si', 'tdk', 'ya',
                      'jd', 'jgn', 'sdh', 'ygy', 'hehe', 'pen',
                      'yah', 'ga', 'ok', 'bgt', 'gw', 'gue',
                      'lu', 'loe', 'km', 'kmu', 'rt', 'tuh'])
list_stopwprd = set(list_stopwords)
def remove_stopwords(words):
  return [word for word in words if word not in list_stopwords]

data['STOP_REMOVAL'] = data['TOKENIZATION'].apply(lambda x: remove_stopwords(x))
data.head(5)

,tweet,Sentimen,clean,cleantweet,TOKENIZATION,STOP_REMOVAL
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif,bit.ly/2L6CcbB? | Seharusnya saat ini Pemerin...,bitlyLCcbB Seharusnya saat ini Pemerintah Lamp...,"[bitlylccbb, seharusnya, saat, ini, pemerintah...","[bitlylccbb, pemerintah, lampung, bantuan, mah..."
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Klo kata Pemerintah tuh lagi menerapkan Herd...,"[klo, kata, pemerintah, tuh, lagi, menerapkan,...","[pemerintah, menerapkan, herd, immunity, kemam..."
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif,Saat ini yang bisa saya lakukan hanya menyiapk...,Saat ini yang bisa saya lakukan hanya menyiapk...,"[saat, ini, yang, bisa, saya, lakukan, hanya, ...","[lakukan, makanan, berbuka, sahur, kepala, kel..."
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif,"emosi bgt, lg kondisi begini gue disuruh liput...",emosi bgt kondisi begini gue disuruh liputan...,"[emosi, bgt, kondisi, begini, gue, disuruh, li...","[emosi, kondisi, disuruh, liputan, covid, dose..."
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif,Bang gimna pemerintah mau peduli RRC urus ABK ...,Bang gimna pemerintah mau peduli RRC urus ABK ...,"[bang, gimna, pemerintah, mau, peduli, rrc, ur...","[bang, gimna, pemerintah, peduli, rrc, urus, a..."


In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in data['STOP_REMOVAL']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

  print(len(term_dict))
  print("....................")

for tern in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  print(term,":" , term_dict[term])

  print(term_dict)
  print("....................")

def get_stemmed_term(document):
  return [term_dict[term] for term in document]

data['Cleantweet'] = data['STOP_REMOVAL'].apply(lambda x: get_stemmed_term(x))

14
....................
32
....................
50
....................
60
....................
76
....................
94
....................
109
....................
111
....................
127
....................
145
....................
149
....................
156
....................
165
....................
169
....................
183
....................
197
....................
198
....................
206
....................
218
....................
232
....................
249
....................
263
....................
269
....................
282
....................
292
....................
303
....................
315
....................
326
....................
337
....................
346
....................
360
....................
380
....................
385
....................
399
....................
402
....................
421
....................
429
....................
446
....................
454
....................
464
....................
472
..

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

